In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from influxdb import InfluxDBClient

In [ ]:
# InfluxDB Connection Details
INFLUX_HOST = 'influxdb'       # The host where InfluxDB is running
INFLUX_PORT = 8086              # The port
INFLUX_USER = 'admin'           # Username
INFLUX_PASS = 'admin123'        # Password
INFLUX_DB   = 'controlled_building' # Name of the BMS database

# Configuration
PE_GAS = 1.00
PE_ELEC = 2.17
COP_H = 0.98
EER_C = 5.4
THRESHOLD = 0  # Power threshold to filter outliers (in Joules)
MEASUREMENT = "simulation_observations" # The measurement containing these fields
YEAR = 2025
# Full year
#START = f"{YEAR}-01-01T00:00:00Z"
#END = f"{YEAR + 1}-01-01T00:00:00Z"
# Summer
START = f"{YEAR}-06-01T00:00:00Z"
END = f"{YEAR}-09-01T00:00:00Z"

# Initialize the Client
client = InfluxDBClient(
    host=INFLUX_HOST,
    port=INFLUX_PORT,
    username=INFLUX_USER,
    password=INFLUX_PASS,
    database=INFLUX_DB
)

assert client.ping(), "❌ Cannot reach InfluxDB"
print("✅ Connected to InfluxDB 1.x")

In [ ]:
def plot_energy_signature(group_by_time, title, threshold):
    """
    Plots the Energy Signature of the building based on InfluxDB data.
    
    Args:
        group_by_time (str): Time interval for grouping data (e.g., '1h', '30m').
        title (str): Title of the plot.
        threshold (float): Minimum power threshold to filter data points.
    
    Returns:
        None
    """
    # Query the specific FMU fields
    influxql_query = f'''
    SELECT last("value") AS "val"
    FROM "{MEASUREMENT}"
    WHERE time >= '{START}'
        AND time < '{END}'
        AND "name" =~ /^(DistrictHeating|DistrictCooling|Electricity|Tair_z1|Tair_z2|Tair_z3|Tair_z4|T_ext)$/
    GROUP BY time({group_by_time}), "name"
    fill(previous)
    '''

    # Query Data and Process into DataFrame
    print(f"Executing query with GROUP BY time({group_by_time})...")
    result_set = client.query(influxql_query)
    points = result_set.get_points()
    df = pd.DataFrame(points)
    print(df)

    if df.empty:
        print("No data returned.")
        return

    # Convert 'time' to datetime
    df['time'] = pd.to_datetime(df['time'])

    # Pivot the DataFrame to have separate columns for each 'name'
    df = df.pivot_table(
        index='time',
        columns='name',
        values='val'
    ).reset_index()

    # Convert to Final Energy
    df['QH_final'] = df['DistrictHeating'].abs() / COP_H
    df['QC_final'] = df['DistrictCooling'].abs() / EER_C
    df['QE_final'] = df['Electricity'].abs()
    
    # Convert to Primary Energy
    df['Q_Total_PE'] = (
        df['QH_final'] * PE_GAS +
        df['QC_final'] * PE_ELEC +
        df['QE_final'] * PE_ELEC
    )
    
    # Average indoor air temperature from zones
    df['T_in_avg'] = (
        df['Tair_z1'] +
        df['Tair_z2'] +
        df['Tair_z3'] +
        df['Tair_z4']
    ) / 4

    # Temperature difference
    df['Delta_T'] = df['T_in_avg'] - df['T_ext']

    # Remove rows with NaN values
    df = df.dropna(subset=['Delta_T', 'Q_Total_PE'])

    # Filter out low power points
    initial_count = len(df)
    df = df[df['Q_Total_PE'] >= threshold]
    removed_count = initial_count - len(df)
    print(f"Filtered {removed_count} points using threshold {threshold} W")

    # Split data into "Summer" (Cooling) and "Winter" (Heating) for coloring
    # Filter by Schedule (Oct-Mar = Heating, Apr-Sep = Cooling)
    month = df['time'].dt.month
    is_heating_season = (month <= 3) | (month >= 10)
    df_winter = df[is_heating_season]
    df_summer = df[~is_heating_season]

    # Scatter points for PE
    plt.figure(figsize=(10, 7))
    plt.scatter(df_summer['Delta_T'], df_summer['Q_Total_PE'], 
                label='Cooling (Summer)', color='tab:blue', alpha=0.3, s=20)
    plt.scatter(df_winter['Delta_T'], df_winter['Q_Total_PE'], 
                label='Heating (Winter)', color='tab:orange', alpha=0.3, s=20)

    # Regression lines
    for data, col, lab in [(df_summer, 'tab:blue', 'Cooling Signature'), 
                        (df_winter, 'tab:orange', 'Heating Signature')]:
        if len(data) > 1:
            z = np.polyfit(data['Delta_T'], data['Q_Total_PE'], 1)
            p = np.poly1d(z)
            sorted_dt = np.sort(data['Delta_T'])
            plt.plot(sorted_dt, p(sorted_dt), color=col, linewidth=3, label=lab)
    
    plt.title(title)
    plt.xlabel('$\Delta T$ (Indoor Avg - Outdoor) [°C]')
    plt.ylabel('Primary Energy Consumption [J]')
    plt.axvline(x=0, color='gray', linestyle='--')
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Plot the hourly energy signature
group_time_hour = '1h'
title_hour = f'Hourly Energy Signature'
plot_energy_signature(group_time_hour, title_hour, THRESHOLD)

# Plot the daily energy signature
group_time_day = '1d'
title_day = f'Daily Energy Signature'
plot_energy_signature(group_time_day, title_day, THRESHOLD)

# Plot the weekly energy signature
group_time_week = '7d'
title_week = f'Weekly Energy Signature'
plot_energy_signature(group_time_week, title_week, THRESHOLD)

# plot the monthly energy signature
group_time_month = '30d'
title_month = f'Monthly Energy Signature'
plot_energy_signature(group_time_month, title_month, THRESHOLD)